In [1]:
import os
if(not os.getcwd().endswith("crchum")):
    os.chdir("..")
print(os.getcwd())
env = ! cat .env
try : 
    DEVICE = [_2.replace("\"", "") for (_1, _2) in [e.split("=") for e in env] if _1 == "DEVICE"][0]
except:
    print("Set the device in .env file")
print("Device : ", DEVICE)
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

/Users/antoinemagron/Documents/EPFL/PDM/crchum
Device :  mps


In [18]:
import datasets

from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from transformers import (AutoModel, AutoTokenizer, LlamaForCausalLM)

import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

FLAG = "epilep"

/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tensor([1.], device='mps:0')


/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Model

In [2]:
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

In [ ]:
tok = AutoTokenizer.from_pretrained("epfl-llm/meditron-7b")
meditron = LlamaForCausalLM.from_pretrained("epfl-llm/meditron-7b")

In [ ]:
meditron = meditron.to(mps_device)
answer = meditron.generate(
    **tok("My patient is epileptic.", return_tensors="pt").to(mps_device)
)

## Datasets

In [2]:
from models import (get_pmc_patients,
                    get_pubmed_ds,
                    get_mimic_iv_notes)

(get_pmc_patients(), 
get_pubmed_ds(split="[:80]", flag_filter=False),
get_mimic_iv_notes())

(Dataset({
     features: ['patient_id', 'patient_uid', 'PMID', 'file_path', 'title', 'patient', 'age', 'gender', 'relevant_articles', 'similar_patients'],
     num_rows: 4129
 }),
 Dataset({
     features: ['title', 'abstract'],
     num_rows: 80
 }),
 Dataset({
     features: ['note_id', 'text'],
     num_rows: 331793
 }))

In [2]:
from models import get_mimic_iv_notes
mimic = get_mimic_iv_notes("disc")

In [4]:
mimic

Dataset({
    features: ['note_id', 'text', '__index_level_0__'],
    num_rows: 12336
})

In [6]:
round(12336 * 100 /331793, 1), "% kept in discharge"

(3.7, '% kept in discharge')

In [9]:
str(type([""]))

"<class 'list'>"